# Machine Learning challenge: Love is love

## Approach
> * Used glob to extract all the paths for the images
> * Used easyocr library to read the text from the .jpg files.
> * Removed **\n** and **contraction** then applied some spelling correction and created Sentiment scores using `SentimentIntensityAnalyzer` & `TextBlob Sentiment polarity score`
> * Removed Non alphabets, Stopwords and applied lemmatization
> * Created TFIDF vectors for the text
> * Created vector represenation for each comment
> * Stacked all these features i.e. `Sentiment scores`, `TFIDF Vectors`, `Comment Vectors` And used these as features to cluster the comments in 3 groups using `Kmeans`.
> * Further by inspection detected the comment types in the cluster and mapped `Random`, `Negative`, `Positive` accordingly.

* *`NOTE` every time we run code we need to inspect the cluster label and mapping string*

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_colwidth", 200) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
# !pip install easyocr

In [ ]:
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
paths=glob.glob('/content/drive/My Drive/Love is Love/Dataset/*')

In [ ]:
paths

['/content/drive/My Drive/Love is Love/Dataset/Test3118.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test1902.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test1992.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test3130.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test3863.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test3625.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test2411.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test2068.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test2977.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test2616.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test1785.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test1634.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test3525.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test3893.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test1872.jpg',
 '/content/drive/My Drive/Love is Love/Dataset/Test2007.jpg',
 '/conte

In [ ]:
import easyocr
reader = easyocr.Reader(['en'])

MD5 hash mismatch, possible file corruption
Re-downloading the recognition model, please wait
Download complete


In [ ]:
x=reader.readtext('/content/drive/My Drive/Love is Love/Dataset/Test1001.jpg')

In [ ]:
x

[([[32, 14], [208, 14], [208, 42], [32, 42]],
  'When people ask',
  0.8024377822875977),
 ([[30, 38], [214, 38], [214, 64], [30, 64]],
  'what Isee in you,',
  0.07900962978601456),
 ([[42, 60], [204, 60], [204, 88], [42, 88]],
  'Ijust smile and',
  0.15037031471729279),
 ([[28, 83], [216, 83], [216, 113], [28, 113]],
  'look away because',
  0.785081684589386),
 ([[8, 104], [248, 104], [248, 134], [8, 134]],
  'Im afraid ifthey knew,',
  0.5500624775886536),
 ([[36, 130], [216, 130], [216, 158], [36, 158]],
  'theyd fall in love',
  0.7659639716148376),
 ([[60, 154], [196, 154], [196, 182], [60, 182]],
  'with you too.',
  0.25999242067337036)]

In [ ]:
text=[]
no=[]
for i in paths:
  data=[]
  for j in reader.readtext(i):
    data.append(j[1])
  text.append(" ".join(data))
  no.append(i.split('\\')[-1])

In [ ]:
text

['KEEP CALM AND LOVE WHO YOU WANT Tar^o kar',
 'Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce',
 "YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar",
 'IE m  ess a to w nores will kn nver m La amei9 can Le zou M> Anela',
 'THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT',
 'orr0 ululce yggy anesAN marriages UEVE PPOPE STII PROTESTIN THIS SHIT Simely because iLis noneol mydamn businesshow others choose to live their infos',
 "ONNE DAY WE WONT HAVE TO COME OUT OF THE CLOSET' WE'LL JUST SAY WE ARE IN LOV= AND THAT WILL ALL THAT MATT Tao mar",
 'Yov ULLoT HAVL rolnut tit hrD tenilve FIFD.',
 'PROUD ll8l',
 'lo belolroid ls co behove Os che erurh were noe crue.',
 'To be afraid is to behave as if the truth were not true. BAYARD RUSTIN',
 '',
 'Do yu realize whar you when you say sa That\'s so gayl"? GAY" HOr Mrcn cno0 no1. tobo',
 'you re not out and proud, yours urte outannabestmontn ine Don spendanolner hiding closet andsnamne ou donina for roudese

In [ ]:
df=pd.DataFrame(zip(text,no))
df.columns=['text','id']

In [ ]:
df.id=df.id.apply(lambda x:x.split('/')[-1])

In [ ]:
df.head()

,text,id
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg


In [ ]:
df.loc[df.id=='Test107.jpg']

,text,id
182,lesbianvenom: lesbianvenom: straighteners aren't worth ur moneyi've been using one for three weeks andi'm still definitelya lesbian living in oll my20 years of this has been byfar mybest joke ever,Test107.jpg


In [ ]:
df.to_csv('/content/drive/My Drive/Love is Love/text_data.csv',index=False)

In [ ]:
# df=pd.read_csv('text_data_dev.csv')

In [ ]:
# df['text']=df['text'].replace(df.text[0],"")

In [ ]:
df.head()

,text,id
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg


In [ ]:
import re
import spacy
from nltk.tokenize import word_tokenize,regexp_tokenize

In [ ]:
def slash_n(text):
    #removing \n
    text=re.sub('\n',' ',text)
    #converting whole string into lowercase
    text=text.lower()
    return text

In [ ]:
df['text_\n']=df['text'].apply(slash_n)

In [ ]:
df.head()

,text,id,text_\n
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt"


In [ ]:
#importing dictonary containing all the contractions and their expander version as their values
from contr import CONTRACTION_MAP

In [ ]:
def contraction(text):
    """
    This function will return the text in  an expanded form which is in common English. It also helps in generalising the tokens
    """
    tokens=text.split()
    tok=[]
    for i in tokens:
        if i in CONTRACTION_MAP.keys():
            tok.append(CONTRACTION_MAP[i])
        else:
            tok.append(i)
    return ' '.join(tok)

In [ ]:
df['text_cont']=df['text_\n'].apply(contraction)

In [ ]:
df.head(10)

,text,id,text_\n,text_cont
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt"
5,orr0 ululce yggy anesAN marriages UEVE PPOPE STII PROTESTIN THIS SHIT Simely because iLis noneol mydamn businesshow others choose to live their infos,Test3625.jpg,orr0 ululce yggy anesan marriages ueve ppope stii protestin this shit simely because ilis noneol mydamn businesshow others choose to live their infos,orr0 ululce yggy anesan marriages ueve ppope stii protestin this shit simely because ilis noneol mydamn businesshow others choose to live their infos
6,ONNE DAY WE WONT HAVE TO COME OUT OF THE CLOSET' WE'LL JUST SAY WE ARE IN LOV= AND THAT WILL ALL THAT MATT Tao mar,Test2411.jpg,onne day we wont have to come out of the closet' we'll just say we are in lov= and that will all that matt tao mar,onne day we wont have to come out of the closet' we will just say we are in lov= and that will all that matt tao mar
7,Yov ULLoT HAVL rolnut tit hrD tenilve FIFD.,Test2068.jpg,yov ullot havl rolnut tit hrd tenilve fifd.,yov ullot havl rolnut tit hrd tenilve fifd.
8,PROUD ll8l,Test2977.jpg,proud ll8l,proud ll8l
9,lo belolroid ls co behove Os che erurh were noe crue.,Test2616.jpg,lo belolroid ls co behove os che erurh were noe crue.,lo belolroid ls co behove os che erurh were noe crue.


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [ ]:
def spell(text):
    text=str(TextBlob(text).correct())
    return text

In [ ]:
df['text_correct']=df['text_cont'].apply(spell)

In [ ]:
df.head()

,text,id,text_\n,text_cont,text_correct
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get"


In [ ]:
# import nltk
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
sia=SentimentIntensityAnalyzer()

In [ ]:
def score(text):
    # text=str(TextBlob(text).correct())
    s=sia.polarity_scores(text)['compound']
#     t=TextBlob(text).sentiment.polarity
#     s=t+s
    return s

In [ ]:
df['score']=df['text_correct'].apply(score)

In [ ]:
df.head()

,text,id,text_\n,text_cont,text_correct,score
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106


In [ ]:
df[df['score']==0]

,text,id,text_\n,text_cont,text_correct,score
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0
6,ONNE DAY WE WONT HAVE TO COME OUT OF THE CLOSET' WE'LL JUST SAY WE ARE IN LOV= AND THAT WILL ALL THAT MATT Tao mar,Test2411.jpg,onne day we wont have to come out of the closet' we'll just say we are in lov= and that will all that matt tao mar,onne day we wont have to come out of the closet' we will just say we are in lov= and that will all that matt tao mar,one day we wont have to come out of the closet' we will just say we are in love= and that will all that matt to may,0.0
7,Yov ULLoT HAVL rolnut tit hrD tenilve FIFD.,Test2068.jpg,yov ullot havl rolnut tit hrd tenilve fifd.,yov ullot havl rolnut tit hrd tenilve fifd.,you allot have walnut tit had senile find.,0.0
11,,Test1634.jpg,,,,0.0
...,...,...,...,...,...,...
234,GT,Test143.jpg,gt,gt,it,0.0
235,,Test158.jpg,,,,0.0
236,,Test225.jpg,,,,0.0
237,,Test228.jpg,,,,0.0


In [ ]:
# import spacy.cli
# spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
nlp=spacy.load('en_core_web_md')

In [ ]:
#Using Spacy's Lemmatization
def lemma(text):
    doc=nlp(text)
    tok=[i.lemma_ for i in doc]
    return ' '.join(tok)

In [ ]:
df['text_lemma']=df['text_correct'].apply(lemma)

In [ ]:
df.head()

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404,keep calm and love who -PRON- want tar^o war,keep calm and love who PRON want tar o kar,2
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,1
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263,-PRON- say ' be straight ' i say taste the rainbow ' to may,PRON say be stfajight i say mtaste the rainbow to mar,2
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000,in m less a to w note will in over m la amid can le you m > anel,ie m ess a to w nore will kn nver m la amei can le zou m anela,0
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106,"think mirriiglisas aripeiigchns miss blind , bltmlinaunn anna don get",thimk mirriiglisas aripeiigchns imiis blin bltmlinaunn an a dnn gtbt,1


In [ ]:
def remove_non_alpha(text):
    tok=regexp_tokenize(text,'[A-Za-z]+')
    tokn=[]
    for i in tok:
        tokn.append(i.strip())
    return ' '.join(tokn)

In [ ]:
df['text_no_pun']=df['text_lemma'].apply(remove_non_alpha)

In [ ]:
df.head()

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404,keep calm and love who -PRON- want tar^o war,keep calm and love who PRON want tar o war,2
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,1
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263,-PRON- say ' be straight ' i say taste the rainbow ' to may,PRON say be straight i say taste the rainbow to may,2
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000,in m less a to w note will in over m la amid can le you m > anel,in m less a to w note will in over m la amid can le you m anel,0
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106,"think mirriiglisas aripeiigchns miss blind , bltmlinaunn anna don get",think mirriiglisas aripeiigchns miss blind bltmlinaunn anna don get,1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(stop_words='english',strip_accents='unicode',min_df=2)

In [ ]:
tfvec=tfidf.fit_transform(df['text_no_pun'])

In [ ]:
tfvec

<239x200 sparse matrix of type '<class 'numpy.float64'>'
	with 707 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import csr_matrix,hstack

In [ ]:
score=csr_matrix(df.score).T

In [ ]:
score

<239x1 sparse matrix of type '<class 'numpy.float64'>'
	with 113 stored elements in Compressed Sparse Column format>

In [ ]:
vec=[]
for i in df.text_no_pun.values:
    vec.append(nlp(i).vector)

In [ ]:
vectors=csr_matrix(vec)

In [ ]:
vectors

<239x300 sparse matrix of type '<class 'numpy.float32'>'
	with 52200 stored elements in Compressed Sparse Row format>

In [ ]:
#created a feature matrix
f_matrix=hstack((score,vectors,tfvec))

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km = KMeans(n_clusters=3)
km.fit(f_matrix)
clusters = km.labels_.tolist()

In [ ]:
df['cluster']=clusters

In [ ]:
df.head()

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404,keep calm and love who -PRON- want tar^o war,keep calm and love who PRON want tar o war,2
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,1
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263,-PRON- say ' be straight ' i say taste the rainbow ' to may,PRON say be straight i say taste the rainbow to may,2
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000,in m less a to w note will in over m la amid can le you m > anel,in m less a to w note will in over m la amid can le you m anel,1
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106,"think mirriiglisas aripeiigchns miss blind , bltmlinaunn anna don get",think mirriiglisas aripeiigchns miss blind bltmlinaunn anna don get,1


In [ ]:
df[df['cluster']==0]

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
11,,Test1634.jpg,,,,0.0,,,0
19,&ozel9e o c6~~#?9 auae Aelo,Test2280.jpg,&ozel9e o c6~~#?9 auae aelo,&ozel9e o c6~~#?9 auae aelo,&ozel9e o c~~#?9 alae felo,0.0,& ozel9e o c~~#?9 alae felo,ozel e o c alae felo,0
21,Tatergreram dnallaar lrn nanllya fnn,Test3566.jpg,tatergreram dnallaar lrn nanllya fnn,tatergreram dnallaar lrn nanllya fnn,tatergreram dnallaar len natalya inn,0.0,tatergreram dnallaar len natalya inn,tatergreram dnallaar len natalya inn,0
31,,Test1279.jpg,,,,0.0,,,0
36,,Test2049.jpg,,,,0.0,,,0
...,...,...,...,...,...,...,...,...,...
233,,Test105.jpg,,,,0.0,,,0
235,,Test158.jpg,,,,0.0,,,0
236,,Test225.jpg,,,,0.0,,,0
237,,Test228.jpg,,,,0.0,,,0


In [ ]:
df[df['cluster']==1]

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,1
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000,in m less a to w note will in over m la amid can le you m > anel,in m less a to w note will in over m la amid can le you m anel,1
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106,"think mirriiglisas aripeiigchns miss blind , bltmlinaunn anna don get",think mirriiglisas aripeiigchns miss blind bltmlinaunn anna don get,1
5,orr0 ululce yggy anesAN marriages UEVE PPOPE STII PROTESTIN THIS SHIT Simely because iLis noneol mydamn businesshow others choose to live their infos,Test3625.jpg,orr0 ululce yggy anesan marriages ueve ppope stii protestin this shit simely because ilis noneol mydamn businesshow others choose to live their infos,orr0 ululce yggy anesan marriages ueve ppope stii protestin this shit simely because ilis noneol mydamn businesshow others choose to live their infos,or ululce gay anesan marriages eve pope stir protesting this shit simply because ilio none madame businesshow others choose to live their into,-0.7935,or ululce gay anesan marriages eve pope stir protest this shit simply because ilio none madame businesshow other choose to live -PRON- into,or ululce gay anesan marriages eve pope stir protest this shit simply because ilio none madame businesshow other choose to live PRON into,1
9,lo belolroid ls co behove Os che erurh were noe crue.,Test2616.jpg,lo belolroid ls co behove os che erurh were noe crue.,lo belolroid ls co behove os che erurh were noe crue.,lo belolroid is co behave os the truth were not true.,-0.0083,lo belolroid be co behave os the truth be not true .,lo belolroid be co behave os the truth be not true,1
...,...,...,...,...,...,...,...,...,...
218,Jason J 0 jaybs121|4| Added Me Everyone add my new Snapchat or comment names below. III be posting positive quotes every day Igbtpride loveislove Igbt trans transmatter transgender transgirl trans...,Test191.jpg,jason j 0 jaybs121|4| added me everyone add my new snapchat or comment names below. iii be posting positive quotes every day igbtpride loveislove igbt trans transmatter transgender transgirl trans...,jason j 0 jaybs121|4| added me everyone add my new snapchat or comment names below. iii be posting positive quotes every day igbtpride loveislove igbt trans transmatter transgender transgirl trans...,jason j 0 jaybs121|4| added me everyone add my new snapchat or comment names below. iii be posting positive quotes every day igbtpride loveislove it trans transmitted transgender transgirl transla...,0.8126,jason j 0 jaybs121|4| add -PRON- everyone add -PRON- new snapchat or comment name below . iii be post positive quote every day igbtpride loveislove -PRON- trans transmit transgender transgirl tran...,jason j jaybs add PRON everyone add PRON new snapchat or comment name below iii be post positive quote every day igbtpride loveislove PRON trans transmit transgender transgirl translate lesion les...,1
219,F HAI,Test211.jpg,f hai,f hai,f had,0.0000,f have,f have,1
228,learned Ihis the hard way... aoueihersyhow nileelandhck upoe siynollungand leo aluck)vu uplnstesds,Test242.jpg,lea

In [ ]:
df[df['cluster']==2]

,text,id,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404,keep calm and love who -PRON- want tar^o war,keep calm and love who PRON want tar o war,2
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263,-PRON- say ' be straight ' i say taste the rainbow ' to may,PRON say be straight i say taste the rainbow to may,2
6,ONNE DAY WE WONT HAVE TO COME OUT OF THE CLOSET' WE'LL JUST SAY WE ARE IN LOV= AND THAT WILL ALL THAT MATT Tao mar,Test2411.jpg,onne day we wont have to come out of the closet' we'll just say we are in lov= and that will all that matt tao mar,onne day we wont have to come out of the closet' we will just say we are in lov= and that will all that matt tao mar,one day we wont have to come out of the closet' we will just say we are in love= and that will all that matt to may,0.0000,one day -PRON- will not have to come out of the closet ' -PRON- will just say -PRON- be in love= and that will all that matt to may,one day PRON will not have to come out of the closet PRON will just say PRON be in love and that will all that matt to may,2
7,Yov ULLoT HAVL rolnut tit hrD tenilve FIFD.,Test2068.jpg,yov ullot havl rolnut tit hrd tenilve fifd.,yov ullot havl rolnut tit hrd tenilve fifd.,you allot have walnut tit had senile find.,0.0000,-PRON- allot have walnut tit have senile find .,PRON allot have walnut tit have senile find,2
8,PROUD ll8l,Test2977.jpg,proud ll8l,proud ll8l,proud all,0.4767,proud all,proud all,2
...,...,...,...,...,...,...,...,...,...
189,Eor cul5 jir,Test231.jpg,eor cul5 jir,eor cul5 jir,for cult air,0.0000,for cult air,for cult air,2
194,cRu 8,Test234.jpg,cru 8,cru 8,cry 8,-0.4767,cry 8,cry,2
221,Married with PRIDF,Test100.jpg,married with pridf,married with pridf,married with pride,0.3400,married with pride,married with pride,2
224,for LGBT,Test161.jpg,for lgbt,for lgbt,for left,0.0000,for left,for left,2


In [ ]:
# df['Category']=df.cluster.map({:'Random',:'Positive',:'Negative'}) ##template

In [ ]:
df['Category']=df.cluster.map({0:'Random',2:'Positive',1:'Negative'})

In [ ]:
df.columns

Index(['text', 'id', 'text_\n', 'text_cont', 'text_correct', 'score',
       'text_lemma', 'text_no_pun', 'cluster', 'Category'],
      dtype='object')

In [ ]:
df.columns=['text', 'Filename', 'text_\n', 'text_cont', 'text_correct', 'score','text_lemma', 'text_no_pun', 'cluster', 'Score']

In [ ]:
df.head()

,text,Filename,text_\n,text_cont,text_correct,score,text_lemma,text_no_pun,cluster,Score
0,KEEP CALM AND LOVE WHO YOU WANT Tar^o kar,Test3118.jpg,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o kar,keep calm and love who you want tar^o war,0.4404,keep calm and love who -PRON- want tar^o war,keep calm and love who PRON want tar o war,2,Positive
1,Turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,Test1902.jpg,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor mnnr aalern irn oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,0.0000,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,turaeayrerou ram anrngnnlor man adler in oarlanlmnnr ce,1,Negative
2,YOU SAY 'BE STFAJIGHT' I SAY MTASTE THE RAINBOW' To Mar,Test1992.jpg,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be stfajight' i say mtaste the rainbow' to mar,you say 'be straight' i say taste the rainbow' to may,0.2263,-PRON- say ' be straight ' i say taste the rainbow ' to may,PRON say be straight i say taste the rainbow to may,2,Positive
3,IE m ess a to w nores will kn nver m La amei9 can Le zou M> Anela,Test3130.jpg,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,ie m ess a to w nores will kn nver m la amei9 can le zou m> anela,in m less a to w notes will in over m la amid can le you m> anel,0.0000,in m less a to w note will in over m la amid can le you m > anel,in m less a to w note will in over m la amid can le you m anel,1,Negative
4,"THIMK MIRRIIGLISAS ArIpeIIGChnS IMIIS BLIN, BLTMLINAunN AN0A DnN GTBT",Test3863.jpg,"thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","thimk mirriiglisas aripeiigchns imiis blin, bltmlinaunn an0a dnn gtbt","think mirriiglisas aripeiigchns miss blind, bltmlinaunn anna don get",-0.5106,"think mirriiglisas aripeiigchns miss blind , bltmlinaunn anna don get",think mirriiglisas aripeiigchns miss blind bltmlinaunn anna don get,1,Negative


In [ ]:
df.shape

(239, 10)

In [ ]:
test=pd.read_csv('/content/drive/My Drive/Love is Love/Test.csv')

In [ ]:
test.head()

,Filename,Category
0,Test1001.jpg,NaN
1,Test1012.jpg,NaN
2,Test1022.jpg,NaN
3,Test1071.jpg,NaN
4,Test1122.jpg,NaN


In [ ]:
test.shape

(239, 2)

In [ ]:
sub=pd.merge(test,df,on='Filename')[['Filename','Score']]

In [ ]:
sub

,Filename,Score
0,Test1001.jpg,Positive
1,Test1012.jpg,Positive
2,Test1022.jpg,Positive
3,Test1071.jpg,Positive
4,Test1122.jpg,Negative
...,...,...
234,Test243.jpg,Negative
235,Test244.jpg,Random
236,Test245.jpg,Negative
237,Test249.jpg,Random


In [ ]:
sub.columns=test.columns

In [ ]:
sub

,Filename,Category
0,Test1001.jpg,Positive
1,Test1012.jpg,Positive
2,Test1022.jpg,Positive
3,Test1071.jpg,Positive
4,Test1122.jpg,Negative
...,...,...
234,Test243.jpg,Negative
235,Test244.jpg,Random
236,Test245.jpg,Negative
237,Test249.jpg,Random


In [ ]:
sub.to_csv('/content/drive/My Drive/Love is Love/submission1.csv',index=False)

<img src='Untitled.png' />